In [1]:
import boto3
import pymongo
import datetime
import json
import io

In [15]:
def list_s3_directory(bucket, path):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket)
    objects = []
    for o in bucket.objects.filter(Prefix = path):
        objects.append(o.key)
    return(objects)

def prep_for_insert(record):
    """Take a sindle record from s3 json and prep for mongodb
    
    - Convert to datetime objects for dates
    - Set '_id' property
    """
    record['_id'] = record['accession']
    for k in ['Received', 'Published', 'LastUpdate', 'LastMetaUpdate']:
        try:
            record[k] = datetime.datetime.strptime(record[k],"%Y-%m-%dT%X.%fZ")
        except:
            pass
        
def insert_to_mongo(collection, dbname = 'sra', drop_first = True):
    client = pymongo.MongoClient()
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('omicidx.cancerdatasci.org')
    db = client[dbname]
    if(drop_first):
        try:
            db[collection].drop()
        except:
            pass
    col = db[collection]
    all_s3_files = filter(lambda z: z.endswith('json'),
                          list_s3_directory('omicidx.cancerdatasci.org', 
                                         path='sra/NCBI_SRA_Mirroring_20180401_Full/json/{}_json/'.format(collection)))
    for i in all_s3_files:
        records = []
        for line in io.StringIO(bucket.Object(i).get()['Body'].read().decode('UTF-8')):
            rec = json.loads(line)
            prep_for_insert(rec)
            records.append(rec)
        col.insert_many(records)



'sra/NCBI_SRA_Mirroring_20180401_Full/fileinfo_addons.csv.gz'

In [97]:
for r in rec:
    fix_dates(r)

In [101]:
from pymongo import MongoClient
client = MongoClient()
db = client.sra

In [105]:
expt = db['experiment']

In [ ]:
insert_to_mongo(client, 'run')

In [16]:
import multiprocessing
with multiprocessing.Pool() as pool:
    pool.map(insert_to_mongo, ['study', 'experiment', 'sample', 'run'])

[None, None, None, None]